In [ ]:
!pip install snowflake-connector-python
!pip install tldextract
!pip install py2neo

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.site.gs.com/simple
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.site.gs.com/simple
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.site.gs.com/simple
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.


In [ ]:
import requests
import pandas as pd
import numpy as np
import snowflake.connector
import os
from py2neo import Graph
from py2neo.bulk import merge_nodes, create_relationships, merge_relationships
from urllib.parse import urlparse

#[Environment Configs]
client_id = "d0afa05e7ab642acb0497f018b601b86"
client_secret = "ikdU3eOtXRmMR9oWlEnP0W0jAVZYNMBF2FUwsuwtHkZ37Oh4IZZkso3tFdsbfdmO"  
os.environ["HTTP_PROXY"] = "http://d161560-001.dc.gs.com:8899"
os.environ["HTTPS_PROXY"] = "http://d161560-001.dc.gs.com:8899"

#[Credentials]
host = "xplore.graph-x.site.gs.com"
port = "7687"
username = "neo4j"
password = "xplorer"
graph = Graph(host=host, port=port, auth=(username, password))
BATCHES = 100


### Libraries & Shared Code

In [ ]:
def get_oauth_tokens(client_id: str, client_secret: str):
    _IDFS_URL = "https://idfs.gs.com/as/token.oauth2"
    response = requests.post(
        url = _IDFS_URL,
        params = {"access_token_manager_id": "JwtSnowflake"},  # This must be set to JwtSnowflake to generate Snowflake-compatible OAuth Token
        data = {"grant_type": "client_credentials", "scope": "SESSION:ROLE-ANY"},
        auth = (client_id, client_secret),
    )
    response = response.json()
    if "access_token" not in response.keys():
        raise PermissionError(f"Not authorized to access OAuth Tokens: {response}")
    return response["access_token"]
  
def connect_snowflake(user: str, database: str, token: str) -> snowflake.connector.cursor:
    """Connect to Snowflake and return an authorized cursor. Below uses Catalyst connections
   
    :param user: Client ID from OAuth2 Authentication
    :param database: Name of database to connect to
    :param token: Generated OAuth Token. Please either generate and save to a tmp file in a separate thread OR use subprocess module to call the underlying subsystem and capture the output
    :return: Snowflake Cursor
    """
    snow_connector = snowflake.connector.connect(
        user=user,
        authenticator="oauth",
        account="sfamdprvtawseast1d01.goldman.us-east-1.aws.privatelink",
        role="AMD_CATALYST_DATA_RW",
        warehouse="AMD_CATALYST_RW",
        database=database,
        ocsp_fail_open=False,
        token=token,
    )
    cursor = snow_connector.cursor()
    return cursor
 
def disconnect(cursor: snowflake.connector.cursor) -> None:
    try:
        cursor.close()
    except:
        raise Exception("Failed to close cursor")   
        
oauth_token = get_oauth_tokens(client_id, client_secret)
cursor = connect_snowflake(client_id, "AMD_CATALYST_DB", oauth_token)

# SourceScrub Person
### Step 3: Creating Nodes

In [ ]:
sourcescrub_persons_df = cursor.execute('SELECT ID, FIRSTNAME, LASTNAME, LINKEDIN, CITY, STATE, COUNTRY FROM L1.SOURCESCRUB_PERSON').fetch_pandas_all()

In [ ]:
display(sourcescrub_persons_df.head(5))

,ID,FIRSTNAME,LASTNAME,LINKEDIN,CITY,STATE,COUNTRY
0,2874625,Craig,Gress,https://www.linkedin.com/in/craiggress,Chinle,Arizona,United States
1,2874624,Eduardo,Resende,https://www.linkedin.com/in/edresende,Sao Paulo,Sao Paulo,Brazil
2,2874623,Miguel,Angel,None,Deba,Euskal Autonomia Erkidegoa,Spain
3,2874622,Brendan,Bateman,https://www.linkedin.com/in/brendanbateman,Sydney,State of New South Wales,Australia
4,2874621,Alena,Bennett,https://www.linkedin.com/in/alenabennett,Sydney,State of New South Wales,Australia


        ID FIRSTNAME LASTNAME                                    LINKEDIN  \
0  2874625     Craig    Gress      https://www.linkedin.com/in/craiggress   
1  2874624   Eduardo  Resende       https://www.linkedin.com/in/edresende   
2  2874623    Miguel    Angel                                        None   
3  2874622   Brendan  Bateman  https://www.linkedin.com/in/brendanbateman   
4  2874621     Alena  Bennett    https://www.linkedin.com/in/alenabennett   

        CITY                       STATE        COUNTRY  
0     Chinle                     Arizona  United States  
1  Sao Paulo                   Sao Paulo         Brazil  
2       Deba  Euskal Autonomia Erkidegoa          Spain  
3     Sydney    State of New South Wales      Australia  
4     Sydney    State of New South Wales      Australia  

In [ ]:
display(sourcescrub_persons_df.head(5))

,ID,LINKEDIN,CITY,STATE,COUNTRY,NAME
0,2874625,https://www.linkedin.com/in/craiggress,Chinle,Arizona,United States,Craig Gress
1,2874624,https://www.linkedin.com/in/edresende,Sao Paulo,Sao Paulo,Brazil,Eduardo Resende
2,2874623,None,Deba,Euskal Autonomia Erkidegoa,Spain,Miguel Angel
3,2874622,https://www.linkedin.com/in/brendanbateman,Sydney,State of New South Wales,Australia,Brendan Bateman
4,2874621,https://www.linkedin.com/in/alenabennett,Sydney,State of New South Wales,Australia,Alena Bennett


        ID                                    LINKEDIN       CITY  \
0  2874625      https://www.linkedin.com/in/craiggress     Chinle   
1  2874624       https://www.linkedin.com/in/edresende  Sao Paulo   
2  2874623                                        None       Deba   
3  2874622  https://www.linkedin.com/in/brendanbateman     Sydney   
4  2874621    https://www.linkedin.com/in/alenabennett     Sydney   

                        STATE        COUNTRY             NAME  
0                     Arizona  United States      Craig Gress  
1                   Sao Paulo         Brazil  Eduardo Resende  
2  Euskal Autonomia Erkidegoa          Spain     Miguel Angel  
3    State of New South Wales      Australia  Brendan Bateman  
4    State of New South Wales      Australia    Alena Bennett  

In [ ]:
sourcescrub_persons_df['LINKEDIN'] = sourcescrub_persons_df['LINKEDIN'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.').strip('/'))

### Ingesting Common SourceScrub Identifiers into Graph

In [ ]:
444968
444968

In [ ]:
# Dividing the DataFrame into smaller chunks for ingestion
sourcescrub_persons_df_linkedin = np.array_split(sourcescrub_persons_df[sourcescrub_persons_df['LINKEDIN'].notna()][['LINKEDIN']].rename(columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)

# Creating nodes for LinkedIn identifiers
for chunk_df in sourcescrub_persons_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "LinkedInURL"), "URL"),
    )

# Creating nodes for Sourcescrub Persons
sourcescrub_persons = np.array_split([{k:v for k, v in x.items() if pd.notnull(v)} for x in sourcescrub_persons_df.to_dict('records')], BATCHES)
for chunk_df in sourcescrub_persons:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("Person", "SourcescrubPerson"), "SOURCE_ID"),
    )

In [ ]:
sourcescrub_persons_df.head(5)

,SOURCE_ID,LINKEDIN,CITY,STATE,COUNTRY,NAME
0,sourcescrub-2874625,linkedin.com/in/craiggress,Chinle,Arizona,United States,Craig Gress
1,sourcescrub-2874624,linkedin.com/in/edresende,Sao Paulo,Sao Paulo,Brazil,Eduardo Resende
2,sourcescrub-2874623,,Deba,Euskal Autonomia Erkidegoa,Spain,Miguel Angel
3,sourcescrub-2874622,linkedin.com/in/brendanbateman,Sydney,State of New South Wales,Australia,Brendan Bateman
4,sourcescrub-2874621,linkedin.com/in/alenabennett,Sydney,State of New South Wales,Australia,Alena Bennett


             SOURCE_ID                        LINKEDIN       CITY  \
0  sourcescrub-2874625      linkedin.com/in/craiggress     Chinle   
1  sourcescrub-2874624       linkedin.com/in/edresende  Sao Paulo   
2  sourcescrub-2874623                                       Deba   
3  sourcescrub-2874622  linkedin.com/in/brendanbateman     Sydney   
4  sourcescrub-2874621    linkedin.com/in/alenabennett     Sydney   

                        STATE        COUNTRY             NAME  
0                     Arizona  United States      Craig Gress  
1                   Sao Paulo         Brazil  Eduardo Resende  
2  Euskal Autonomia Erkidegoa          Spain     Miguel Angel  
3    State of New South Wales      Australia  Brendan Bateman  
4    State of New South Wales      Australia    Alena Bennett  

### Ingesting Common SourceScrub Identifiers into Graph

In [ ]:
sourcescrub_persons_df_linkedin = np.array_split(sourcescrub_persons_df[sourcescrub_persons_df['LINKEDIN'].isna() == False][['LINKEDIN']].rename(columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_persons_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "LinkedInURL"), "URL"),
    )

In [ ]:
sourcescrub_persons =np.array_split([{k:v for k, v in x.items() if v == v} for x in sourcescrub_persons_df.to_dict('records')], BATCHES)
for chunk_df in sourcescrub_persons:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("Person", "SourcescrubPerson"), "SOURCE_ID"),
    )

In [ ]:
sourcescrub_identifier_relationships = sourcescrub_persons_df.loc[sourcescrub_persons_df['LINKEDIN'] != np.nan][['SOURCE_ID', 'LINKEDIN']]
sourcescrub_identifier_relationships.insert(1, 'WEIGHT', [[0.5]] * len(sourcescrub_identifier_relationships))
df_list = np.array_split(sourcescrub_identifier_relationships, BATCHES)
for chunk_df in df_list:
        merge_relationships(
            graph.auto(),
            chunk_df.values.tolist(),
            merge_key="LINKED_TO",
            start_node_key=(("Person", "SourcescrubPerson"), "SOURCE_ID"),
            end_node_key=(("URLIdentifier", "LinkedInURL"), "URL"),
            keys=["WEIGHT"]
        )

## Person to Company

In [1]:
sourcescrub_p2c_df = cursor.execute('SELECT PERSONID ,COMPANYID, STARTDATE, ENDDATE FROM L1.SOURCESCRUB_PERSONTOCOMPANY').fetch_pandas_all()
sourcescrub_p2c_df.head()

NameError: NameError: name 'cursor' is not defined

In [ ]:
sourcescrub_p2c_df.rename(columns={'PERSONID': 'PERSON_ID', 'COMPANYID': 'COMPANY_ID', 'STARTDATE': 'START_DATE', 'ENDDATE': 'END_DATE'}, inplace=True)
sourcescrub_p2c_df.START_DATE = sourcescrub_p2c_df.START_DATE.fillna(0).astype('str').replace('0', np.nan)
sourcescrub_p2c_df.END_DATE = sourcescrub_p2c_df.END_DATE.fillna(0).astype('str').replace('0', np.nan)
sourcescrub_p2c_df.head()

In [ ]:
sourcescrub_curr_p2c_df = sourcescrub_p2c_df[sourcescrub_p2c_df['END_DATE'] .isna() == True]
print(sourcescrub_curr_p2c_df.shape)


sourcescrub_prev_p2c_df = sourcescrub_p2c_df[sourcescrub_p2c_df['END_DATE'].isna() == False]
print(sourcescrub_prev_p2c_df.shape)

In [ ]:
sourcescrub_identifier_relationships = sourcescrub_prev_p2c_df.loc[sourcescrub_prev_p2c_df['LINKEDIN'] != np.nan][['SOURCE_ID', 'LINKEDIN']]
sourcescrub_identifier_relationships.insert(1, 'WEIGHT', [[0.5]] * len(sourcescrub_identifier_relationships))
df_list = np.array_split(sourcescrub_identifier_relationships, BATCHES)

for chunk_df in df_list:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="PREVIOUST_EMPLOYMENT",
        start_node_key=(("Person", "SourcescrubPerson"), "PERSON_ID"),
        end_node_key=(("Company", "SourcescrubCompany"), "COMPANY_ID"),
        keys=["WEIGHT"]
        )
    

In [ ]:
# Filter for rows where CURRENT is True
sourcescrub_identifier_relationships = sourcescrub_curr_p2c_df.loc[sourcescrub_curr_p2c_df['LINKEDIN'] != np.nan][['SOURCE_ID', 'LINKEDIN']]
sourcescrub_p2c_current_relationships = sourcescrub_curr_p2c_df[['PERSON_ID', 'COMPANY_ID']]
sourcescrub_p2c_current_relationships.insert(1, 'WEIGHT', [[0.5]] * len(sourcescrub_p2c_current_relationships))
df_list = np.array_split(sourcescrub_p2c_current_relationships, BATCHES)

for chunk_df in df_list:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="CURRENT_EMPLOYMENT",
        start_node_key=(("Person", "SourcescrubPerson"), "PERSON_ID"),
        end_node_key=(("Company", "SourcescrubCompany"), "COMPANY_ID"),
        keys=["WEIGHT"]
    )

In [ ]:
# Step 4: Creating Relationships
# Creating relationships between SourceScrub persons and their LinkedIn URLs
sourcescrub_identifier_relationships = sourcescrub_persons_df[sourcescrub_persons_df['LINKEDIN'].notna()][['SOURCE_ID', 'LINKEDIN']].copy()
sourcescrub_identifier_relationships.insert(1, 'WEIGHT', [[0.5]] * len(sourcescrub_identifier_relationships))
df_list = np.array_split(sourcescrub_identifier_relationships, BATCHES)

for chunk_df in df_list:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="LINKED_TO",
        start_node_key=(("Person", "SourcescrubPerson"), "SOURCE_ID"),
        end_node_key=(("URLIdentifier", "LinkedInURL"), "LINKEDIN"),
        keys=["WEIGHT"]
    )

# Step 5: Mapping Identifiers
# Preparing relationships between persons and companies
sourcescrub_p2c_df = cursor.execute('SELECT PERSONID ,COMPANYID, STARTDATE, ENDDATE FROM L1.SOURCESCRUB_PERSONTOCOMPANY').fetch_pandas_all()



# Step 5: Mapping Identifiers
# Preparing relationships between persons and companies
sourcescrub_p2c_df.rename(columns={'PERSONID': 'PERSON_ID', 'COMPANYID': 'COMPANY_ID', 'STARTDATE': 'START_DATE', 'ENDDATE': 'END_DATE'}, inplace=True)

# Creating relationships for previous employment
sourcescrub_prev_p2c_df = sourcescrub_p2c_df[sourcescrub_p2c_df.END_DATE.notna()]
df_list = np.array_split(sourcescrub_prev_p2c_df[['PERSON_ID', 'COMPANY_ID']], BATCHES)

for chunk_df in df_list:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="PREVIOUS_EMPLOYMENT",
        start_node_key=(("Person", "SourcescrubPerson"), "PERSON_ID"),
        end_node_key=(("Company", "SourcescrubCompany"), "COMPANY_ID"),
        keys=[]
    )

# Creating relationships for current employment
sourcescrub_curr_p2c_df = sourcescrub_p2c_df[sourcescrub_p2c_df.END_DATE.isna()]
df_list = np.array_split(sourcescrub_curr_p2c_df[['


### Crunchbase Persons

In [ ]:
crunchbase_persons_df = cursor.execute('SELECT UUID, "CB URL", NAME, "TWITTER", "FACEBOOK", "LINKEDIN URL", CITY, COUNTRY_CODE FROM L1.CRUNCHBASE_PEOPLE').fetch_pandas_all()

In [ ]:
crunchbase_persons_df['UUID'] = 'crunchbase-' + crunchbase_persons_df['UUID'].astype(str)
crunchbase_persons_df.rename(columns={'UUID': 'SOURCE_ID', 'CB URL': 'CRUNCHBASE', 'LINKEDIN URL': 'LINKEDIN'}, inplace=True)
display(crunchbase_persons_df.head(5))

In [ ]:
crunchbase_persons_df['SOURCE_ID'] = 'sourcescrub-' + crunchbase_persons_df['SOURCE_ID']

crunchbase_persons_df['LINKEDIN'] = crunchbase_persons_df['LINKEDIN'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.').strip('/'))
crunchbase_persons_df['TWITTER'] = crunchbase_persons_df['TWITTER'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.').strip('/'))
crunchbase_persons_df['FACEBOOK'] = crunchbase_persons_df['FACEBOOK'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.').strip('/'))
crunchbase_persons_df['CRUNCHBASE'] = crunchbase_persons_df['CRUNCHBASE'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.').strip('/'))

#sourcescrub_organizations_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [ ]:
crunchbase_persons_df.head()

,SOURCE_ID,NAME,DOMAIN,LINKEDIN,TWITTER,FACEBOOK,CRUNCHBASE,TIME_OF_FOUNDING,CITY,STATE,COUNTRY,SPECIALTIES,TOTAL_AMOUNT_INVESTED
0,sourcescrub-284125,"Titanium Partners, LLC",titaniumpartnersllc.com,linkedin.com/company/titanium-partners,NaN,NaN,NaN,2012.0,Sydney,State of New South Wales,Australia,"executive search, operations & supply chain, t...",NaN
1,sourcescrub-284124,"Domtec International, LLC",domtec.com,linkedin.com/company/domtec-international,twitter.com/domtecint,facebook.com/domtecinternational,NaN,1995.0,Idaho Falls,Idaho,United States,"monolithic concrete bulk storage domes, thin-s...",NaN
2,sourcescrub-284123,"All Mountain Technologies, LLC",allmtntech.com,linkedin.com/company/all-mountain-technologies...,twitter.com/allmtntech,facebook.com/allmtntech,crunchbase.com/organization/all-mountain-techn...,2003.0,Edwards,Colorado,United States,"it and network managed services, wireless netw...",NaN
3,sourcescrub-284122,Value Store It Self Storage,valuestoreit.com,linkedin.com/company/value-store-it,twitter.com/valuestoreit,facebook.com/valuestoreit,NaN,2001.0,Miami,Florida,United States,"self storage, public storage, rv storage, auto...",NaN
4,sourcescrub-284121,Simsolid Corp.,altairhyperworks.com,linkedin.com/company/6440741,twitter.com/simsolid,facebook.com/simsolid.corporation,crunchbase.com/organization/simsolid,2015.0,Orange,California,United States,"structural simulation software, it software, c...",0.0


            SOURCE_ID                            NAME  \
0  sourcescrub-284125          Titanium Partners, LLC   
1  sourcescrub-284124       Domtec International, LLC   
2  sourcescrub-284123  All Mountain Technologies, LLC   
3  sourcescrub-284122     Value Store It Self Storage   
4  sourcescrub-284121                  Simsolid Corp.   

                    DOMAIN                                           LINKEDIN  \
0  titaniumpartnersllc.com             linkedin.com/company/titanium-partners   
1               domtec.com          linkedin.com/company/domtec-international   
2           allmtntech.com  linkedin.com/company/all-mountain-technologies...   
3         valuestoreit.com                linkedin.com/company/value-store-it   
4     altairhyperworks.com                       linkedin.com/company/6440741   

                    TWITTER                           FACEBOOK  \
0                       NaN                                NaN   
1     twitter.com/domtecint   facebook

### Ingesting Common CB Identifiers into Graph

In [ ]:
crunchbase_persons_df_linkedin = np.array_split(crunchbase_persons_df[crunchbase_persons_df['LINKEDIN'].isna() == False][['LINKEDIN']].rename(columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_persons_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "LinkedInURL"), "URL"),
    )
    
crunchbase_persons_df_df_twitter = np.array_split(crunchbase_persons_df[crunchbase_persons_df['TWITTER'].isna() == False][['TWITTER']].rename(columns={'TWITTER': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_persons_df_df_twitter:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "TwitterURL"), "URL"),
    )

crunchbase_persons_df_facebook =  np.array_split(crunchbase_persons_df[crunchbase_persons_df['FACEBOOK'].isna() == False][['FACEBOOK']].rename(columns={'FACEBOOK': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_persons_df_facebook:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "FacebookURL"), "URL"),
    )
    
crunchbase_persons_df_crunchbase =  np.array_split(crunchbase_persons_df[crunchbase_persons_df['CRUNCHBASE'].isna() == False][['CRUNCHBASE']].rename(columns={'CRUNCHBASE': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_persons_df_crunchbase:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "CrunchbaseURL"), "URL"),
    )

### Ingesting SourceScrub Companies into the Graph

In [ ]:
crunchbase_persons = np.array_split([{k:v for k, v in x.items() if v == v} for x in crunchbase_persons_df.to_dict('records')], BATCHES)
for chunk_df in crunchbase_persons:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("Person", "CrunchbasePerson"), "SOURCE_ID"),
    )

In [ ]:
for identifier in ['LINKEDIN', 'TWITTER', 'FACEBOOK', 'CRUNCHBASE']:
    cb_identifier_relationships = crunchbase_persons_df.loc[crunchbase_persons_df[identifier] != np.nan][['SOURCE_ID', identifier]]
    cb_identifier_relationships.insert(1, 'WEIGHT', [[0.5]] * len(cb_identifier_relationships))

    df_list = np.array_split(cb_identifier_relationships, BATCHES)
    for chunk_df in df_list:
        merge_relationships(
            graph.auto(),
            chunk_df.values.tolist(),
            merge_key="LINKED_TO",
            start_node_key=("PERSON", "SOURCE_ID"),
            end_node_key=("URLIdentifier", "URL"),
            keys=["WEIGHT"],
        )

### CrunchBase

In [ ]:
crunchbase_organizations_df = cursor.execute('SELECT CRUNCHBASE_ORGANIZATION_UUID, ORGANIZATION_CITY, FACEBOOK_URL, DOMAIN, TWITTER_URL, FUNDING_ROUND_TYPE, FOUNDED_ON, EMPLOYEE_COUNT, CRUNCHBASE_GROUPS, ORGANIZATION_STATE_CODE, ORGANIZATION_NAME, LINKEDIN_URL, ORGANIZATION_COUNTRY_CODE, CRUNCHBASE_URL FROM L2_NEW.GRAPH_CRUNCHBASE_ORGANIZATIONS WHERE DOMAIN IS NOT NULL OR TWITTER_URL IS NOT NULL OR FACEBOOK_URL IS NOT NULL OR LINKEDIN_URL IS NOT NULL OR CRUNCHBASE_URL IS NOT NULL').fetch_pandas_all()
crunchbase_organizations_df.rename(columns={
    'CRUNCHBASE_ORGANIZATION_UUID': 'SOURCE_ID', 
    'TWITTER_URL': 'TWITTER', 
    'FACEBOOK_URL': 'FACEBOOK',
    'LINKEDIN_URL': 'LINKEDIN',
    'CRUNCHBASE_URL': 'CRUNCHBASE', 
    'FOUNDED_ON': 'TIME_OF_FOUNDING',
    'ORGANIZATION_CITY': 'CITY',
    'ORGANIZATION_STATE_CODE': 'STATE',
    'ORGANIZATION_COUNTRY_CODE': 'COUNTRY', 
    'ORGANIZATION_NAME': 'NAME', 
    'LINKEDIN_URL': 'LINKEDIN'
}, inplace=True)

crunchbase_organizations_df.replace({None: ''}, inplace=True)

crunchbase_organizations_df['LINKEDIN'] = crunchbase_organizations_df['LINKEDIN'].apply(lambda x: (str(urlparse(x.replace('[', '').replace(']', '')).netloc or '') + str(urlparse(x.replace('[', '').replace(']', '')).path or '')).lstrip('www.').strip('/'))
crunchbase_organizations_df['TWITTER'] = crunchbase_organizations_df['TWITTER'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.').strip('/'))
crunchbase_organizations_df['FACEBOOK'] = crunchbase_organizations_df['FACEBOOK'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.').strip('/'))
crunchbase_organizations_df['CRUNCHBASE'] = crunchbase_organizations_df['CRUNCHBASE'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.').strip('/'))

crunchbase_organizations_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
display(crunchbase_organizations_df.head())

,SOURCE_ID,CITY,FACEBOOK,DOMAIN,TWITTER,FUNDING_ROUND_TYPE,TIME_OF_FOUNDING,EMPLOYEE_COUNT,CRUNCHBASE_GROUPS,STATE,NAME,LINKEDIN,COUNTRY,CRUNCHBASE
0,crunchbase-b883e57f-5a42-4cce-81f3-b70ad9482052,New York,NaN,infinity.xyz,twitter.com/infinitydotxyz,NaN,NaN,unknown,"Financial Services,Lending and Investments,Pay...",NY,Infinity,NaN,USA,crunchbase.com/organization/infinity-2052
1,crunchbase-57eeadb3-5173-4fd9-a9f8-dec1ad472417,New York,facebook.com/pages/category/Insurance-company/...,skyviewagency.com,NaN,NaN,NaN,1-10,Financial Services,NY,SkyView Agency,linkedin.com/company/skyview-agency,USA,crunchbase.com/organization/skyview-agency
2,crunchbase-f81e2ff9-6a87-4629-8a7e-230855a0dd30,Toshima,NaN,muio.co.jp,NaN,NaN,1991-02-01,11-50,"Hardware,Information Technology,Internet Servi...",NaN,Muio Creative,NaN,JPN,crunchbase.com/organization/muio-creative
3,crunchbase-ebff3a2b-1b34-4bc8-8b46-5640352c99c0,Suva,facebook.com/vititalk,vititalk.com,twitter.com/viti_talk,NaN,NaN,unknown,"Hardware,Messaging and Telecommunications",NaN,Viti Talk Telecommunications,NaN,FJI,crunchbase.com/organization/viti-talk-telecomm...
4,crunchbase-2774d086-6884-4d8b-9968-58181ed4e104,Vienna,NaN,membershipcards.com,NaN,NaN,1995-01-01,11-50,"Content and Publishing,Design,Media and Entert...",NaN,Membership Cards Only,linkedin.com/company/membership-cards-only-llc,AUT,crunchbase.com/organization/membership-cards-only


                                         SOURCE_ID      CITY  \
0  crunchbase-b883e57f-5a42-4cce-81f3-b70ad9482052  New York   
1  crunchbase-57eeadb3-5173-4fd9-a9f8-dec1ad472417  New York   
2  crunchbase-f81e2ff9-6a87-4629-8a7e-230855a0dd30   Toshima   
3  crunchbase-ebff3a2b-1b34-4bc8-8b46-5640352c99c0      Suva   
4  crunchbase-2774d086-6884-4d8b-9968-58181ed4e104    Vienna   

                                            FACEBOOK               DOMAIN  \
0                                                NaN         infinity.xyz   
1  facebook.com/pages/category/Insurance-company/...    skyviewagency.com   
2                                                NaN           muio.co.jp   
3                              facebook.com/vititalk         vititalk.com   
4                                                NaN  membershipcards.com   

                      TWITTER FUNDING_ROUND_TYPE TIME_OF_FOUNDING  \
0  twitter.com/infinitydotxyz                NaN              NaN   
1             

In [ ]:
crunchbase_organizations_df_website = np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['DOMAIN'].isna() == False][['DOMAIN']].rename(columns={'DOMAIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_website:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "DomainURL"), "URL"),
    )

crunchbase_organizations_df_linkedin = np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['LINKEDIN'].isna() == False][['LINKEDIN']].rename(columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "LinkedInURL"), "URL"),
    )
    
crunchbase_organizations_df_twitter = np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['TWITTER'].isna() == False][['TWITTER']].rename(columns={'TWITTER': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_twitter:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "TwitterURL"), "URL"),
    )

crunchbase_organizations_df_facebook =  np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['FACEBOOK'].isna() == False][['FACEBOOK']].rename(columns={'FACEBOOK': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_facebook:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "FacebookURL"), "URL"),
    )

crunchbase_organizations_df_crunchbase =  np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['CRUNCHBASE'].isna() == False][['CRUNCHBASE']].rename(columns={'CRUNCHBASE': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_crunchbase:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "CrunchbaseURL"), "URL"),
    )

In [ ]:
crunchbase_organizations = np.array_split([{k:v for k, v in x.items() if v == v} for x in crunchbase_organizations_df.to_dict('records')], BATCHES)
for chunk_df in crunchbase_organizations:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("Company", "CrunchbaseCompany"), "SOURCE_ID"),
    )

In [ ]:
for identifier in ['DOMAIN', 'LINKEDIN', 'TWITTER', 'FACEBOOK', 'CRUNCHBASE']:
    crunchbase_identifier_relationships = crunchbase_organizations_df.loc[crunchbase_organizations_df[identifier] != np.nan][['SOURCE_ID', identifier]]
    crunchbase_identifier_relationships.insert(1, 'WEIGHT', [[0.5]] * len(crunchbase_identifier_relationships))

    df_list = np.array_split(crunchbase_identifier_relationships, BATCHES)
    for chunk_df in df_list:
        merge_relationships(
            graph.auto(),
            chunk_df.values.tolist(),
            merge_key="LINKED_TO",
            start_node_key=("Company", "SOURCE_ID"),
            end_node_key=("URLIdentifier", "URL"),
            keys=["WEIGHT"],
        )

In [ ]:
# Relationship creation for previous employment
sourcescrub_prev_p2c_df_filtered = sourcescrub_prev_p2c_df.loc[sourcescrub_prev_p2c_df['LINKEDIN'] != np.nan]
df_list = np.array_split(sourcescrub_prev_p2c_df_filtered[['PERSON_ID', 'COMPANY_ID']], BATCHES)

for chunk_df in df_list:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="PREVIOUST_EMPLOYMENT",
        start_node_key=(("Person", "SourcescrubPerson"), "PERSON_ID"),
        end_node_key=(("Company", "SourcescrubCompany"), "COMPANY_ID"),
        keys=["WEIGHT"]
    )

# Relationship creation for current employment
sourcescrub_curr_p2c_df_filtered = sourcescrub_curr_p2c_df.loc[sourcescrub_curr_p2c_df['LINKEDIN'] != np.nan]
df_list = np.array_split(sourcescrub_curr_p2c_df_filtered[['PERSON_ID', 'COMPANY_ID']], BATCHES)

for chunk_df in df_list:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="CURRENT_EMPLOYMENT",
        start_node_key=(("Person", "SourcescrubPerson"), "PERSON_ID"),
        end_node_key=(("Company", "SourcescrubCompany"), "COMPANY_ID"),
        keys=["WEIGHT"]
    )

# Relationship creation for LinkedIn URLs
sourcescrub_identifier_relationships = sourcescrub_curr_p2c_df_filtered[['SOURCE_ID', 'LINKEDIN']]
df_list = np.array_split(sourcescrub_identifier_relationships, BATCHES)

for chunk_df in df_list:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="LINKED_TO",
        start_node_key=(("Person", "SourcescrubPerson"), "SOURCE_ID"),
        end_node_key=(("URLIdentifier", "LinkedInURL"), "LINKEDIN"),
        keys=["WEIGHT"]
    )

In [ ]:

# Step 4: Creating Relationships
# Creating relationships between SourceScrub persons and their LinkedIn URLs
sourcescrub_identifier_relationships = sourcescrub_persons_df[sourcescrub_persons_df['LINKEDIN'].notna()][['SOURCE_ID', 'LINKEDIN']].copy()
sourcescrub_identifier_relationships.insert(1, 'WEIGHT', [[0.5]] * len(sourcescrub_identifier_relationships))
df_list = np.array_split(sourcescrub_identifier_relationships, BATCHES)

for chunk_df in df_list:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="LINKED_TO",
        start_node_key=(("Person", "SourcescrubPerson"), "SOURCE_ID"),
        end_node_key=(("URLIdentifier", "LinkedInURL"), "LINKEDIN"),
        keys=["WEIGHT"]
    )

≈

In [ ]:

# Step 4: Creating Relationships
# Creating relationships between SourceScrub persons and their LinkedIn URLs
sourcescrub_identifier_relationships = sourcescrub_persons_df[sourcescrub_persons_df['LINKEDIN'].notna()][['SOURCE_ID', 'LINKEDIN']].copy()
sourcescrub_identifier_relationships.insert(1, 'WEIGHT', [[0.5]] * len(sourcescrub_identifier_relationships))
df_list = np.array_split(sourcescrub_identifier_relationships, BATCHES)

for chunk_df in df_list:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="LINKED_TO",
        start_node_key=(("Person", "SourcescrubPerson"), "SOURCE_ID"),
        end_node_key=(("URLIdentifier", "LinkedInURL"), "LINKEDIN"),
        keys=["WEIGHT"]
    )

# Step 5: Mapping Identifiers
# Preparing relationships between persons and companies
sourcescrub_p2c_df = cursor.execute('SELECT PERSONID ,COMPANYID, STARTDATE, ENDDATE FROM L1.SOURCESCRUB_PERSONTOCOMPANY').fetch_pandas_all()
sourcescrub_p2c_df.rename(columns={'PERSONID': 'PERSON_ID', 'COMPANYID': 'COMPANY_ID', 'STARTDATE': 'START_DATE', 'ENDDATE': 'END_DATE'}, inplace=True)

# Creating relationships for previous employment
sourcescrub_prev_p2c_df = sourcescrub_p2c_df[sourcescrub_p2c_df.END_DATE.notna()]
df_list = np.array_split(sourcescrub_prev_p2c_df[['PERSON_ID', 'COMPANY_ID']], BATCHES)

for chunk_df in df_list:
    merge_relationships(
        graph.auto(),
        chunk_df.values.tolist(),
        merge_key="PREVIOUS_EMPLOYMENT",
        start_node_key=(("Person", "SourcescrubPerson"), "PERSON_ID"),
        end_node_key=(("Company", "SourcescrubCompany"), "COMPANY_ID"),
        keys=[]
    )